This notebook performs ANOVA analysis on the 20% data split from 60 - 20 - 20 train-ANOVA-test split obtained from xgb notebook. By reading the 9959301 kmer features into chunks of size 30000, and use ANOVA to get the top 10 features in each chunk, the notebook ends with selected top 3320 features for gradient-boosted model and SVM training.

In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, f_classif
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import eli5

In [2]:
print(1)

1


In [3]:
chunks = pd.read_csv("ANOVA_before_transpose.csv", chunksize = 40000)

In [4]:
df_species = pd.read_parquet('final_df_species.parquet')
df_species = df_species.T
df_species.rename(columns={0: 'species'}, inplace=True)

In [5]:
df_species

,species
GCF_900446045.1_58932_D01.genomic.fna.gz,Brucella abortus
GCF_005938105.1_ASM593810v1.genomic.fna.gz,Brucella haematophila
GCF_000413955.1_Bruc_abor_biovar_1_B10-0091_V1.genomic.fna.gz,Brucella abortus
GCF_022024335.1_ASM2202433v1.genomic.fna.gz,Brucella pseudogrignonensis
GCF_002191835.1_ASM219183v1.genomic.fna.gz,Brucella melitensis
...,...
GCF_000698245.1_ASM69824v1.genomic.fna.gz,Brucella suis
GCF_003993895.1_ASM399389v1.genomic.fna.gz,Brucella melitensis
GCF_023651035.1_ASM2365103v1.genomic.fna.gz,Brucella abortus
GCF_017177355.1_ASM1717735v1.genomic.fna.gz,Brucella melitensis


In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [7]:
# class MultiColumnLabelEncoder:
#     def __init__(self,columns = None):
#         self.columns = columns # array of column names to encode

#     def fit(self,X,y=None):
#         return self # not relevant here

#     def transform(self,X):
#         '''
#         Transforms columns of X specified in self.columns using
#         LabelEncoder(). If no columns specified, transforms all
#         columns in X.
#         '''
#         output = X.copy()
#         if self.columns is not None:
#             for col in self.columns:
#                 output[col] = LabelEncoder().fit_transform(output[col])
#         else:
#             for colname,col in output.iteritems():
#                 output[colname] = LabelEncoder().fit_transform(col)
#         return output

#     def fit_transform(self,X,y=None):
#         return self.fit(X,y).transform(X)

In [8]:
# df_species_enc = MultiColumnLabelEncoder(columns = ['species']).fit_transform(df_species)
# df_species_enc

In [9]:
selected_features = []

In [10]:
for chunk in chunks:
    column_name = chunk.columns.values.tolist()[0]
    chunk = chunk.set_index(column_name)
    chunk.index.name = None
    X = chunk.T
    checker = X
    data = X.join(df_species)
    x_data = data.drop(columns=["species"])
    y_data = data["species"]
    pipe_final = make_pipeline(XGBClassifier(eval_metric = 'mlogloss', random_state = 123, max_depth = 9, n_estimators = 50))
    pipe_final.fit(x_data,y_data)
    eli5_table = eli5.formatters.as_dataframe.explain_weights_df(pipe_final.named_steps["xgbclassifier"])
    features = list(eli5_table.values[:10,0])
    score_checker = eli5_table
    for i in features:
        selected_features.append(i)
    break

/hobbitholes/waymoot/hchen/micromamba/envs/ach.0.8/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [11]:
selected_features

['ATTTTAGATGCTCCTCAAGACGGGC',
 'GACGCTCGTTGTCTTCTTCGGCGCA',
 'AAAAGAGCCGAACCATCTTCCAATC',
 'AGCTTCTCGACCTGATCGAGAGCCT',
 'ATCTCTAAAGACTCGCACATCACAA',
 'ATCCGACCAGAGTGGAACGAGAACA',
 'ATCCTGCTCCGGCAAAACCTCCTTC',
 'CGGTGGAACGCCTACTCCTTTTGGA',
 'CCTGCCCGTGCTTGAGCTTGGGCCG',
 'CTGGTGATCGACCTCGTGGTAGCGA']

In [12]:
np.shape(eli5_table[eli5_table["weight"] != 0])

(364, 2)

In [13]:
eli5_table[eli5_table["weight"] != 0]

,feature,weight
0,ATTTTAGATGCTCCTCAAGACGGGC,0.311563
1,GACGCTCGTTGTCTTCTTCGGCGCA,0.253261
2,AAAAGAGCCGAACCATCTTCCAATC,0.068675
3,AGCTTCTCGACCTGATCGAGAGCCT,0.023459
4,ATCTCTAAAGACTCGCACATCACAA,0.023459
...,...,...
359,CTTGCCGCGTGAAACCGAATAAAAC,0.000013
360,AAAACTGATAATGCCCGCCGTTTCC,0.000013
361,TGATGTCGCCAAGGCGCTCGTCGCA,0.000004
362,GGATAAACGGCTTCCAGCGTGAAAC,0.000003


In [14]:
eli5_table

,feature,weight
0,ATTTTAGATGCTCCTCAAGACGGGC,0.311563
1,GACGCTCGTTGTCTTCTTCGGCGCA,0.253261
2,AAAAGAGCCGAACCATCTTCCAATC,0.068675
3,AGCTTCTCGACCTGATCGAGAGCCT,0.023459
4,ATCTCTAAAGACTCGCACATCACAA,0.023459
...,...,...
39995,AATAGGGCAGTAAGGCAATAAGGCA,0.000000
39996,GCGATGGCCGTGACATTCAGCACGC,0.000000
39997,CGGCGCCCATCAGTCCGAGAAAACG,0.000000
39998,AAAACAGAACCAGCGGCACCAGGGC,0.000000


In [15]:
print(selected_features)

['ATTTTAGATGCTCCTCAAGACGGGC', 'GACGCTCGTTGTCTTCTTCGGCGCA', 'AAAAGAGCCGAACCATCTTCCAATC', 'AGCTTCTCGACCTGATCGAGAGCCT', 'ATCTCTAAAGACTCGCACATCACAA', 'ATCCGACCAGAGTGGAACGAGAACA', 'ATCCTGCTCCGGCAAAACCTCCTTC', 'CGGTGGAACGCCTACTCCTTTTGGA', 'CCTGCCCGTGCTTGAGCTTGGGCCG', 'CTGGTGATCGACCTCGTGGTAGCGA']


In [16]:
# final_selected_features